## Import libraries


In [1]:
import numpy as np
import pandas as pd
from random import choice, uniform, random, randint
from operator import itemgetter, attrgetter
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Activation,Flatten,Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras import backend
from tensorflow.keras.applications import VGG16, MobileNet
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from IPython.display import Image, display
import random
import math
import tensorflow as tf
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model 
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Activation,Concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import backend, models
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import VGG16, MobileNet
from keras.applications.vgg16 import preprocess_input

In [2]:
population = []
noPop = 6
fmap = [16,32,64,128,256]
kernel = [2,3,5,7,9]
activation = ['relu','sigmoid','tanh']
dropout = [0.0,0.1,0.2,0.3,0.4,0.5]
learningRate = [0.0001,0.0005, 0.001, 0.005, 0.01,0.05,0.1,0.5] 
mutationRate = 0.3 
noGen = 10 

## Preparation and analysis of dataset

In [3]:
np.random.seed(12)
train = ImageDataGenerator(rescale=1./255).flow_from_directory('dataset/train',target_size=(224,224))

Found 35215 images belonging to 250 classes.


## Genetic Algorithm

In [4]:
def initializingPopulation(population,noPop):
    for i in range (0,noPop):
        loss = 0.0
        accuracy = 0.0
        inPop = [choice(fmap),choice(kernel),choice(activation),choice(dropout),choice(learningRate),loss,accuracy]
        population.append (inPop)

def rankSelection (population,noPop): 
    population = sorted(population,key=itemgetter(5))
    population = sorted(population,key=itemgetter(6),reverse=True)
    noHalfPopulation = noPop//2 
    halfPopulation = population[:noHalfPopulation]

    return halfPopulation

def tournamentSelection(selectedRank,tSize):
    best = None 
    for i in range(0,tSize):
        ind = choice(selectedRank)
        if best == None:
            best = ind
        elif best == ind:
            ind = choice(selectedRank)

        if best[5]>ind[5] and best[6]<ind[6]:
            best = ind

    return best

def crossover(p1,p2): 
    crossoverPoint = len(p1)//2 
    c1 = p1[:crossoverPoint] + p2[crossoverPoint:]
    c2 = p2[:crossoverPoint] + p1[crossoverPoint:]
    return c1, c2

def mutation (c1,c2):
    rate = random.random()
    print(c1)
    print(c2)
    
    print ('rate: ' +str(rate))
    if rate<=mutationRate:
        length = len(c1)-2
        print ('length: '+str(length))
        index = randint(0,length)
        print('index: ' +str(index))

        if index == 0:
            c1[index] = choice(fmap)
            c2[index] = choice(fmap)
        elif index == 1:
            c1[index] = choice(kernel)
            c2[index] = choice(kernel)
        elif index == 2:
            c1[index] = choice(activation)
            c2[index] = choice(activation)
        elif index == 3:
            c1[index] = choice(dropout)
            c2[index] = choice(dropout)
        elif index == 4:
            c1[index] = choice(learningRate)
            c2[index] = choice(learningRate)
    return c1,c2

In [5]:
def cnn(population):
    for p in population:
        #this is where the fitness value will evaluate
        f = p[0] # features map variable
        k = p[1] # kernel variable
        a = p[2] # activation function variable
        d = p[3] # dropout variable
        l = p[4] # learning rate variable
        print(f,k,a,d,l)
        
        backend.clear_session()

        base_mobilenet = MobileNet(weights = 'imagenet', include_top = False, 
                           input_shape = (224,224,3))
        base_mobilenet.trainable = False # Freeze the mobilenet weights.

        model = Sequential()
        model.add(base_mobilenet)
        model.add(Conv2D(f,(k,k),input_shape=(224,224,3),padding='same'))
        model.add(Activation(a))
        model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
        model.add(Flatten())
        model.add(Dense(32*32))
        model.add(Dropout(d))
        model.add(Dense(250))
        model.add(Activation('softmax'))

        model.compile(Adam(l),loss='categorical_crossentropy',metrics=['accuracy'])

        history = model.fit(train,epochs=100,steps_per_epoch=10,verbose=2)
        acc = np.mean(history.history['accuracy'])
        loss = np.mean(history.history['loss'])
        print(acc)
        print(loss)

        p[5] = loss
        p[6] = acc
        print(p[5])
        print(p[6])


initializingPopulation(population,noPop)

print ('Generation 0')
for p in population:
    print(p)

for gen in range(0,noGen):

    cnn(population)

    selectedRank = rankSelection(population,noPop)
    print('Selected Ranked Chromosomes:')
    for sr in selectedRank:
        print(sr)

    if selectedRank[-1][5]<6 and selectedRank[-1][6]>0.6:
        # this is  stopping condition for genetic algorithm.
        # you can change or delete it if you want
        # but, remember, waiting for 10 generation is time consumption
        # and also computationally overhead
        break
    else:
        child = [] 
        tSize = 2 
        numberOfMating = len(selectedRank)//2
        # this is limiter  for number of child will generate
        # let say, you have 12 total population
        # let say, you have 6 ranking selection
        # the mating process will be 3 time
        # but, since the generated child is 2
        # thus mating process would generate 6 children
        # this is because number of mating * generated Children = half new population
        # 3 mating process * 2 newly generated child = 6 population
        for i in range (0,numberOfMating):
            #this is where mating process take place
            parent1 = tournamentSelection(selectedRank,tSize)
            parent2 = tournamentSelection(selectedRank,tSize)
            while parent2 is parent1:
                parent2 = tournamentSelection(selectedRank,tSize)
                #this is to make sure the candidate does not identical
            c1,c2 = crossover(parent1,parent2)
            print('Child 1: %s child 2: %s'%(c1,c2))
            c1,c2 = mutation(c1,c2)
            child.append(c1)
            child.append(c2)

        population.clear() 
        population = selectedRank + child 

        print ('New Child' + str(gen))
        for c in child:
            print(c)

        print('New Generation ' + str(gen+1))
        for p in population:
            print(p)

Generation 0
[64, 2, 'sigmoid', 0.5, 0.001, 0.0, 0.0]
[256, 2, 'tanh', 0.5, 0.001, 0.0, 0.0]
[256, 9, 'tanh', 0.0, 0.5, 0.0, 0.0]
[256, 5, 'tanh', 0.3, 0.1, 0.0, 0.0]
[32, 9, 'relu', 0.3, 0.01, 0.0, 0.0]
[128, 7, 'sigmoid', 0.2, 0.1, 0.0, 0.0]
64 2 sigmoid 0.5 0.001
Epoch 1/100
10/10 - 10s - loss: 6.3604 - accuracy: 0.0031
Epoch 2/100
10/10 - 8s - loss: 5.7967 - accuracy: 0.0000e+00
Epoch 3/100
10/10 - 7s - loss: 5.6206 - accuracy: 0.0063
Epoch 4/100
10/10 - 8s - loss: 5.6348 - accuracy: 0.0125
Epoch 5/100
10/10 - 7s - loss: 5.5890 - accuracy: 0.0063
Epoch 6/100
10/10 - 7s - loss: 5.5827 - accuracy: 0.0031
Epoch 7/100
10/10 - 7s - loss: 5.5150 - accuracy: 0.0094
Epoch 8/100
10/10 - 7s - loss: 5.5435 - accuracy: 0.0094
Epoch 9/100
10/10 - 7s - loss: 5.5472 - accuracy: 0.0156
Epoch 10/100
10/10 - 7s - loss: 5.4900 - accuracy: 0.0188
Epoch 11/100
10/10 - 7s - loss: 5.5431 - accuracy: 0.0156
Epoch 12/100
10/10 - 7s - loss: 5.4589 - accuracy: 0.0094
Epoch 13/100
10/10 - 7s - loss: 5.4651 - 

10/10 - 7s - loss: 5.3178 - accuracy: 0.3531
Epoch 37/100
10/10 - 8s - loss: 5.2959 - accuracy: 0.3438
Epoch 38/100
10/10 - 7s - loss: 5.0830 - accuracy: 0.3750
Epoch 39/100
10/10 - 7s - loss: 4.5381 - accuracy: 0.4031
Epoch 40/100
10/10 - 7s - loss: 4.3717 - accuracy: 0.4219
Epoch 41/100
10/10 - 7s - loss: 4.7286 - accuracy: 0.3719
Epoch 42/100
10/10 - 7s - loss: 4.5048 - accuracy: 0.4062
Epoch 43/100
10/10 - 7s - loss: 4.6196 - accuracy: 0.4156
Epoch 44/100
10/10 - 7s - loss: 4.8405 - accuracy: 0.4156
Epoch 45/100
10/10 - 8s - loss: 4.3166 - accuracy: 0.4625
Epoch 46/100
10/10 - 7s - loss: 4.2705 - accuracy: 0.4875
Epoch 47/100
10/10 - 7s - loss: 4.3365 - accuracy: 0.4375
Epoch 48/100
10/10 - 9s - loss: 4.4058 - accuracy: 0.4375
Epoch 49/100
10/10 - 9s - loss: 4.1087 - accuracy: 0.4750
Epoch 50/100
10/10 - 9s - loss: 4.7971 - accuracy: 0.4469
Epoch 51/100
10/10 - 9s - loss: 4.8531 - accuracy: 0.4281
Epoch 52/100
10/10 - 8s - loss: 4.5996 - accuracy: 0.4344
Epoch 53/100
10/10 - 8s - l

Epoch 67/100
10/10 - 24s - loss: 10025777.0000 - accuracy: 0.0031
Epoch 68/100
10/10 - 22s - loss: 7969865.5000 - accuracy: 0.0094
Epoch 69/100
10/10 - 22s - loss: 8080796.0000 - accuracy: 0.0063
Epoch 70/100
10/10 - 22s - loss: 8234127.0000 - accuracy: 0.0000e+00
Epoch 71/100
10/10 - 22s - loss: 8739195.0000 - accuracy: 0.0063
Epoch 72/100
10/10 - 22s - loss: 9626519.0000 - accuracy: 0.0000e+00
Epoch 73/100
10/10 - 25s - loss: 8618994.0000 - accuracy: 0.0000e+00
Epoch 74/100
10/10 - 23s - loss: 9706897.0000 - accuracy: 0.0031
Epoch 75/100
10/10 - 22s - loss: 8829172.0000 - accuracy: 0.0063
Epoch 76/100
10/10 - 22s - loss: 10446041.0000 - accuracy: 0.0094
Epoch 77/100
10/10 - 23s - loss: 9065094.0000 - accuracy: 0.0000e+00
Epoch 78/100
10/10 - 23s - loss: 8875594.0000 - accuracy: 0.0031
Epoch 79/100
10/10 - 22s - loss: 7656745.5000 - accuracy: 0.0031
Epoch 80/100
10/10 - 24s - loss: 7449342.5000 - accuracy: 0.0063
Epoch 81/100
10/10 - 24s - loss: 8144105.0000 - accuracy: 0.0000e+00
Epo

Epoch 93/100
10/10 - 10s - loss: 100315.7969 - accuracy: 0.0063
Epoch 94/100
10/10 - 10s - loss: 101340.8594 - accuracy: 0.0125
Epoch 95/100
10/10 - 11s - loss: 90734.9922 - accuracy: 0.0031
Epoch 96/100
10/10 - 12s - loss: 86125.7891 - accuracy: 0.0063
Epoch 97/100
10/10 - 11s - loss: 81472.7891 - accuracy: 0.0156
Epoch 98/100
10/10 - 10s - loss: 90496.3984 - accuracy: 0.0063
Epoch 99/100
10/10 - 10s - loss: 88869.7969 - accuracy: 0.0094
Epoch 100/100
10/10 - 10s - loss: 95442.4844 - accuracy: 0.0031
0.005000000102445484
88517.8793359375
88517.8793359375
0.005000000102445484
32 9 relu 0.3 0.01
Epoch 1/100
10/10 - 10s - loss: 34.5584 - accuracy: 0.0125
Epoch 2/100
10/10 - 9s - loss: 5.6380 - accuracy: 0.0000e+00
Epoch 3/100
10/10 - 9s - loss: 5.5630 - accuracy: 0.0031
Epoch 4/100
10/10 - 9s - loss: 5.5872 - accuracy: 0.0063
Epoch 5/100
10/10 - 9s - loss: 5.7195 - accuracy: 0.0000e+00
Epoch 6/100
10/10 - 9s - loss: 5.5556 - accuracy: 0.0219
Epoch 7/100
10/10 - 9s - loss: 5.6056 - accura

Epoch 27/100
10/10 - 12s - loss: 1866.4189 - accuracy: 0.0031
Epoch 28/100
10/10 - 11s - loss: 1989.2494 - accuracy: 0.0063
Epoch 29/100
10/10 - 10s - loss: 1925.3802 - accuracy: 0.0031
Epoch 30/100
10/10 - 10s - loss: 2109.8594 - accuracy: 0.0094
Epoch 31/100
10/10 - 10s - loss: 1774.9395 - accuracy: 0.0063
Epoch 32/100
10/10 - 10s - loss: 2112.0222 - accuracy: 0.0031
Epoch 33/100
10/10 - 10s - loss: 2292.2903 - accuracy: 0.0063
Epoch 34/100
10/10 - 10s - loss: 2885.8552 - accuracy: 0.0125
Epoch 35/100
10/10 - 10s - loss: 2698.8875 - accuracy: 0.0031
Epoch 36/100
10/10 - 10s - loss: 2492.3188 - accuracy: 0.0094
Epoch 37/100
10/10 - 10s - loss: 2419.5913 - accuracy: 0.0063
Epoch 38/100
10/10 - 10s - loss: 2277.0391 - accuracy: 0.0031
Epoch 39/100
10/10 - 10s - loss: 2298.4341 - accuracy: 0.0000e+00
Epoch 40/100
10/10 - 10s - loss: 3047.9487 - accuracy: 0.0063
Epoch 41/100
10/10 - 10s - loss: 2641.6572 - accuracy: 0.0094
Epoch 42/100
10/10 - 10s - loss: 3015.5801 - accuracy: 0.0000e+00


...

Epoch 59/100
10/10 - 7s - loss: 1.9509 - accuracy: 0.5063
Epoch 60/100
10/10 - 7s - loss: 1.9204 - accuracy: 0.5250
Epoch 61/100
10/10 - 7s - loss: 1.7600 - accuracy: 0.5531
Epoch 62/100
10/10 - 6s - loss: 1.9445 - accuracy: 0.5063
Epoch 63/100
10/10 - 6s - loss: 1.9465 - accuracy: 0.5250
Epoch 64/100
10/10 - 7s - loss: 2.0366 - accuracy: 0.4812
Epoch 65/100
10/10 - 6s - loss: 1.7650 - accuracy: 0.5719
Epoch 66/100
10/10 - 7s - loss: 1.8107 - accuracy: 0.5031
Epoch 67/100
10/10 - 7s - loss: 1.5717 - accuracy: 0.6156
Epoch 68/100
10/10 - 6s - loss: 1.7349 - accuracy: 0.5625
Epoch 69/100
10/10 - 6s - loss: 1.7696 - accuracy: 0.5469
Epoch 70/100
10/10 - 7s - loss: 1.8292 - accuracy: 0.5750
Epoch 71/100
10/10 - 6s - loss: 1.5587 - accuracy: 0.5719
Epoch 72/100
10/10 - 7s - loss: 2.0983 - accuracy: 0.5156
Epoch 73/100
10/10 - 7s - loss: 1.7764 - accuracy: 0.5813
Epoch 74/100
10/10 - 6s - loss: 1.7923 - accuracy: 0.5562
Epoch 75/100
10/10 - 7s - loss: 1.9951 - accuracy: 0.4750
Epoch 76/100
1

Epoch 81/100
10/10 - 6s - loss: 1.7731 - accuracy: 0.5375
Epoch 82/100
10/10 - 6s - loss: 1.7831 - accuracy: 0.5469
Epoch 83/100
10/10 - 6s - loss: 1.9093 - accuracy: 0.5094
Epoch 84/100
10/10 - 6s - loss: 1.7445 - accuracy: 0.5594
Epoch 85/100
10/10 - 6s - loss: 1.8066 - accuracy: 0.5437
Epoch 86/100
10/10 - 6s - loss: 1.8419 - accuracy: 0.5656
Epoch 87/100
10/10 - 6s - loss: 1.6577 - accuracy: 0.5844
Epoch 88/100
10/10 - 7s - loss: 1.7500 - accuracy: 0.5875
Epoch 89/100
10/10 - 6s - loss: 1.6174 - accuracy: 0.5719
Epoch 90/100
10/10 - 6s - loss: 1.5037 - accuracy: 0.6062
Epoch 91/100
10/10 - 7s - loss: 1.6376 - accuracy: 0.5906
Epoch 92/100
10/10 - 6s - loss: 1.6602 - accuracy: 0.5656
Epoch 93/100
10/10 - 6s - loss: 1.6689 - accuracy: 0.5938
Epoch 94/100
10/10 - 6s - loss: 1.4207 - accuracy: 0.6375
Epoch 95/100
10/10 - 6s - loss: 1.5571 - accuracy: 0.5781
Epoch 96/100
10/10 - 6s - loss: 1.6249 - accuracy: 0.5562
Epoch 97/100
10/10 - 6s - loss: 1.4507 - accuracy: 0.6250
Epoch 98/100
1

10/10 - 7s - loss: 3.7641 - accuracy: 0.1969
Epoch 20/100
10/10 - 6s - loss: 3.7601 - accuracy: 0.1906
Epoch 21/100
10/10 - 6s - loss: 3.5793 - accuracy: 0.2250
Epoch 22/100
10/10 - 7s - loss: 3.3507 - accuracy: 0.2469
Epoch 23/100
10/10 - 6s - loss: 3.1215 - accuracy: 0.2969
Epoch 24/100
10/10 - 6s - loss: 3.1192 - accuracy: 0.2812
Epoch 25/100
10/10 - 7s - loss: 3.1627 - accuracy: 0.2375
Epoch 26/100
10/10 - 6s - loss: 2.9047 - accuracy: 0.3094
Epoch 27/100
10/10 - 6s - loss: 3.1239 - accuracy: 0.3125
Epoch 28/100
10/10 - 6s - loss: 2.8273 - accuracy: 0.3250
Epoch 29/100
10/10 - 6s - loss: 2.8341 - accuracy: 0.3313
Epoch 30/100
10/10 - 7s - loss: 2.7919 - accuracy: 0.3187
Epoch 31/100
10/10 - 6s - loss: 2.7999 - accuracy: 0.3031
Epoch 32/100
10/10 - 6s - loss: 2.8085 - accuracy: 0.3125
Epoch 33/100
10/10 - 6s - loss: 2.6903 - accuracy: 0.3594
Epoch 34/100
10/10 - 6s - loss: 2.4782 - accuracy: 0.3938
Epoch 35/100
10/10 - 6s - loss: 2.5145 - accuracy: 0.3750
Epoch 36/100
10/10 - 7s - l

Epoch 59/100
10/10 - 6s - loss: 2.1855 - accuracy: 0.4437
Epoch 60/100
10/10 - 6s - loss: 2.0033 - accuracy: 0.5063
Epoch 61/100
10/10 - 6s - loss: 2.0663 - accuracy: 0.4906
Epoch 62/100
10/10 - 6s - loss: 1.6775 - accuracy: 0.5875
Epoch 63/100
10/10 - 6s - loss: 2.1058 - accuracy: 0.4875
Epoch 64/100
10/10 - 6s - loss: 1.8207 - accuracy: 0.5500
Epoch 65/100
10/10 - 6s - loss: 1.9067 - accuracy: 0.5156
Epoch 66/100
10/10 - 6s - loss: 1.8433 - accuracy: 0.5344
Epoch 67/100
10/10 - 7s - loss: 1.7524 - accuracy: 0.5562
Epoch 68/100
10/10 - 6s - loss: 1.8399 - accuracy: 0.5594
Epoch 69/100
10/10 - 6s - loss: 1.6527 - accuracy: 0.5562
Epoch 70/100
10/10 - 7s - loss: 1.7779 - accuracy: 0.5437
Epoch 71/100
10/10 - 6s - loss: 1.9661 - accuracy: 0.5156
Epoch 72/100
10/10 - 6s - loss: 1.6162 - accuracy: 0.5781
Epoch 73/100
10/10 - 6s - loss: 1.7495 - accuracy: 0.5625
Epoch 74/100
10/10 - 6s - loss: 1.7813 - accuracy: 0.5469
Epoch 75/100
10/10 - 6s - loss: 1.8822 - accuracy: 0.5281
Epoch 76/100
1

Epoch 99/100
10/10 - 6s - loss: 1.3382 - accuracy: 0.6438
Epoch 100/100
10/10 - 7s - loss: 1.3859 - accuracy: 0.6406
0.409781250001397
2.6004033851623536
2.6004033851623536
0.409781250001397
Selected Ranked Chromosomes:
[16, 2, 'tanh', 0.5, 0.001, 2.6004033851623536, 0.409781250001397]
[16, 2, 'tanh', 0.5, 0.001, 2.613929785490036, 0.40974999997066336]
[16, 2, 'tanh', 0.5, 0.001, 2.614882788658142, 0.4069167682295665]
Child 1: [16, 2, 'tanh', 0.5, 0.001, 2.6004033851623536, 0.409781250001397] child 2: [16, 2, 'tanh', 0.5, 0.001, 2.613929785490036, 0.40974999997066336]
[16, 2, 'tanh', 0.5, 0.001, 2.6004033851623536, 0.409781250001397]
[16, 2, 'tanh', 0.5, 0.001, 2.613929785490036, 0.40974999997066336]
rate: 0.02105898445900245
length: 5
index: 4
New Child9
[16, 2, 'tanh', 0.5, 0.1, 2.6004033851623536, 0.409781250001397]
[16, 2, 'tanh', 0.5, 0.01, 2.613929785490036, 0.40974999997066336]
New Generation 10
[16, 2, 'tanh', 0.5, 0.001, 2.6004033851623536, 0.409781250001397]
[16, 2, 'tanh', 0

In [6]:
print(population)

[[16, 2, 'tanh', 0.5, 0.001, 2.6004033851623536, 0.409781250001397], [16, 2, 'tanh', 0.5, 0.001, 2.613929785490036, 0.40974999997066336], [16, 2, 'tanh', 0.5, 0.001, 2.614882788658142, 0.4069167682295665], [16, 2, 'tanh', 0.5, 0.1, 2.6004033851623536, 0.409781250001397], [16, 2, 'tanh', 0.5, 0.01, 2.613929785490036, 0.40974999997066336]]


# Tanto fazia mas o cromossoma que escolhi foi:

[16, 2, 'tanh', 0.5, 0.01, 2.6004033851623536, 0.409781250001397]

Será corrido, no ficheiro "CNN-with-GA.ipynb", o model.fit com estes parâmetros. 
Pequenas notas:
- O que difere entre os cromossomas resultantes foi apenas o LR, sendo assim escolhi o que tinha maior accuracy (que é quase insignificante)
- No entanto, no modelo CNN do ficheiro mencionado acima, coloquei uma callback chamada ReduceLROnPlateau na função de fitness que vai reduzindo o LR quando a accuracy para de melhorar. O LR passou de 0.01 -> 0.0005. É uma grande diferença mas podia ter usado um dos outras cromossomas resultantes (que têm todos praticamente os mesmos valores de loss e accuracy) com LR de 0.001 que a diferença já seria menor.
- Também consigo concluir que, apesar de ter colocado valores como 0.0005 nas possibilidades para as combinações dos parâmetros, este não foi escolhido pelo algoritmo inicialmente, o que leva a pensar que, dada outra combinação de valores iniciais, o meu resultado poderia ter sido melhor, não só para o LR como para todos os outros.